In [17]:
import datetime
import pandas as pd 
import math as m
import statsmodels.formula.api as sm
import numpy as np
from data_process_final import gdp_annual, pop_annual, precip_annual, lk_lvl_annual, swe_annual

final = gdp_annual.merge(pop_annual, on = 'date', how = 'outer').merge(precip_annual, on = 'date', how = 'outer').merge(lk_lvl_annual, on = 'date', how = 'outer').merge(swe_annual, on = 'date', how = 'outer')
final['pop_100k'] = final['population']/100000
final['gdp_100k'] = final['gdp']/100000

In [18]:
final_test = final.drop(final.index[25:])
correlation = final_test.corr()
final_test.head(5)

,date,gdp,population,precipitation,lake_level,swe,pop_100k,gdp_100k
0,1997-01-20,57684.0,2065397.0,14.66,4197.613,87.893226,20.65397,0.576840
1,1998-01-20,62296.1,2100562.0,22.18,4199.542,68.533907,21.00562,0.622961
2,1999-01-20,66019.3,2129836.0,12.18,4202.026,55.209665,21.29836,0.660193
3,2000-01-20,70295.9,2244502.0,14.17,4202.126,39.952550,22.44502,0.702959
4,2001-01-20,73057.6,2283715.0,14.26,4200.665,44.135878,22.83715,0.730576


In [19]:
first_reg = sm.ols('lake_level ~ swe ', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.039
Method:                 Least Squares   F-statistic:                   0.09221
Date:                Thu, 20 Apr 2023   Prob (F-statistic):              0.764
Time:                        18:04:46   Log-Likelihood:                -64.285
No. Observations:                  25   AIC:                             132.6
Df Residuals:                      23   BIC:                             135.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4195.9951      1.951   2150.964      0.000    4191.960    4200.031
swe           -0.0107      0.035     -0.304      0.764      -0.084       0.062
==============================================================================
Omnibus:                        2.227   Durbin-Watson:                   0.211
Prob(Omnibus):                  0.328   Jarque-Bera (JB):                1.924
Skew:                           0.632   Prob(JB):                        0.382
Kurtosis:                       2.502   Cond. No.                         164.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
first_reg = sm.ols('lake_level ~ precipitation', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.043
Method:                 Least Squares   F-statistic:                   0.01580
Date:                Thu, 20 Apr 2023   Prob (F-statistic):              0.901
Time:                        18:04:46   Log-Likelihood:                -64.326
No. Observations:                  25   AIC:                             132.7
Df Residuals:                      23   BIC:                             135.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      4195.8064      3.007   1395.404      0.000    4189.586    4202.027
precipitation    -0.0264      0.210     -0.126      0.901      -0.461       0.409
==============================================================================
Omnibus:                        2.367   Durbin-Watson:                   0.238
Prob(Omnibus):                  0.306   Jarque-Bera (JB):                2.032
Skew:                           0.656   Prob(JB):                        0.362
Kurtosis:                       2.519   Cond. No.                         65.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
first_reg = sm.ols('lake_level ~ population', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.682
Model:                            OLS   Adj. R-squared:                  0.668
Method:                 Least Squares   F-statistic:                     49.36
Date:                Thu, 20 Apr 2023   Prob (F-statistic):           3.69e-07
Time:                        18:04:46   Log-Likelihood:                -50.008
No. Observations:                  25   AIC:                             104.0
Df Residuals:                      23   BIC:                             106.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4213.9570      2.662   1582.827      0.000    4208.450    4219.464
population -6.882e-06    9.8e-07     -7.025      0.000   -8.91e-06   -4.86e-06
==============================================================================
Omnibus:                        1.892   Durbin-Watson:                   0.794
Prob(Omnibus):                  0.388   Jarque-Bera (JB):                1.334
Skew:                           0.333   Prob(JB):                        0.513
Kurtosis:                       2.085   Cond. No.                     1.94e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.94e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
first_reg = sm.ols('lake_level ~ gdp', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     37.51
Date:                Thu, 20 Apr 2023   Prob (F-statistic):           3.01e-06
Time:                        18:04:46   Log-Likelihood:                -52.243
No. Observations:                  25   AIC:                             108.5
Df Residuals:                      23   BIC:                             110.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4202.0205      1.150   3655.345      0.000    4199.642    4204.399
gdp         -5.39e-05    8.8e-06     -6.125      0.000   -7.21e-05   -3.57e-05
==============================================================================
Omnibus:                        2.568   Durbin-Watson:                   0.684
Prob(Omnibus):                  0.277   Jarque-Bera (JB):                1.516
Skew:                           0.326   Prob(JB):                        0.469
Kurtosis:                       1.985   Cond. No.                     3.68e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.68e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
first_reg = sm.ols('lake_level ~ swe + precipitation + gdp_100k', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.689
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     15.49
Date:                Thu, 20 Apr 2023   Prob (F-statistic):           1.53e-05
Time:                        18:04:47   Log-Likelihood:                -49.747
No. Observations:                  25   AIC:                             107.5
Df Residuals:                      21   BIC:                             112.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      4205.8315      2.310   1820.851      0.000    4201.028    4210.635
swe              -0.0367      0.023     -1.593      0.126      -0.085       0.011
precipitation    -0.0962      0.136     -0.708      0.487      -0.379       0.187
gdp_100k         -5.8470      0.860     -6.797      0.000      -7.636      -4.058
==============================================================================
Omnibus:                        1.421   Durbin-Watson:                   0.518
Prob(Omnibus):                  0.491   Jarque-Bera (JB):                0.997
Skew:                           0.175   Prob(JB):                        0.607
Kurtosis:                       2.086   Cond. No.                         351.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""